In [38]:
import requests
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from geopy.distance import vincenty

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [37]:
spray = pd.read_csv("/Users/christophermarker/Documents/DSI7/Data/Project_4/west_nile/input/spray.csv")

In [4]:
test = pd.read_csv("/Users/christophermarker/Documents/DSI7/Data/Project_4/west_nile/input/test.csv")

In [272]:
train = pd.read_csv("/Users/christophermarker/Documents/DSI7/Data/Project_4/west_nile/input/train.csv")

In [6]:
weather = pd.read_csv("/Users/christophermarker/Documents/DSI7/Data/Project_4/west_nile/input/weather.csv")

In [21]:
spray.Date.value_counts()

2013-08-15    2668
2013-08-29    2302
2013-07-17    2202
2011-09-07    2114
2013-07-25    1607
2013-08-22    1587
2013-08-08    1195
2013-09-05     924
2013-08-16     141
2011-08-29      95
Name: Date, dtype: int64

In [35]:
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,lat_long
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,"(41.95469, -87.800991)"
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,"(41.95469, -87.800991)"
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0,"(41.994991, -87.769279)"
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0,"(41.974089, -87.824812)"
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0,"(41.974089, -87.824812)"


In [27]:
train['lat_long'] = list(zip(train['Latitude'], train['Longitude']))

In [70]:
our_spot = (42.004805,-87.817460)

In [271]:
train['distance_2_spray'] = train['lat_long'].apply(lambda row: vincenty(row, our_spot).miles)

In [72]:
train['near_spray_spot'] = [1 if i <= 1 else 0 for i in train['distance_2_spray']]

In [73]:
train['near_spray_spot'].value_counts()

0    10390
1      116
Name: near_spray_spot, dtype: int64

In [74]:
keepmetest = train[train['near_spray_spot'] == 1]

In [78]:
keepmetest['NumMosquitos'].describe()

count    116.000000
mean      13.836207
std       15.615457
min        1.000000
25%        2.000000
50%        8.000000
75%       18.000000
max       50.000000
Name: NumMosquitos, dtype: float64

In [82]:
keepmetest['DayRunner']=pd.to_datetime(keepmetest['Date']).dt.dayofyear


In [85]:
keepmetest['Datetime'] = pd.to_datetime(keepmetest['Date'])

In [86]:
keepmetest['Year']=keepmetest['Date'].str.split('-').str[0].astype(int)
keepmetest=keepmetest[keepmetest['Year']==2013]
keepmetest.head(2)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,lat_long,distance_2_spray,near_spray_spot,Year,DayRunner,Datetime
8188,2013-06-07,"7100 North Harlem Avenue, Chicago, IL 60631, USA",CULEX PIPIENS/RESTUANS,71,N HARLEM AVE,T233,"7100 N HARLEM AVE, Chicago, IL",42.009876,-87.807277,9,1,0,"(42.009876, -87.807277)",0.630276,1,2013,158,2013-06-07
8308,2013-06-14,"7100 North Harlem Avenue, Chicago, IL 60631, USA",CULEX PIPIENS/RESTUANS,71,N HARLEM AVE,T233,"7100 N HARLEM AVE, Chicago, IL",42.009876,-87.807277,9,17,0,"(42.009876, -87.807277)",0.630276,1,2013,165,2013-06-14


In [105]:
def analyze_spray(train, spray, spray_date_list, day_runner_ref_list, dist=1):
    ref_counter = 0
    for date in spray_date_list:
        spray_temp = spray[spray['Date'] == date]
        train_temp = train[train['Year'] == int(date.split('-')[0])]
        lat_long_list = list(spray_temp.lat_long.values)
            for lat_long in lat_long_list:
                train_temp['distance_to_spray'] = train_temp['lat_long'].apply(lambda row: vincenty(row, lat_long).miles)
                train_temp_2 = train_temp[train_temp['distance_to_spray'] <= dist]
                train_temp_2['days_removed'] = train_temp_2['DayRunner'].apply(lambda x:x - day_runner_ref_list[ref_counter])
                lat_long_list_2 = list(train_temp_2.lat_long.values)
                for lat_long_2 in lat_long_list_2:
                    
                for i in list(range(-10, 41)):
                    

In [ ]:
spray_day = 

In [153]:
keepmetest

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,lat_long,distance_2_spray,near_spray_spot,Year,DayRunner,Datetime
8188,2013-06-07,"7100 North Harlem Avenue, Chicago, IL 60631, USA",CULEX PIPIENS/RESTUANS,71,N HARLEM AVE,T233,"7100 N HARLEM AVE, Chicago, IL",42.009876,-87.807277,9,1,0,"(42.009876, -87.807277)",0.630276,1,2013,158,2013-06-07
8308,2013-06-14,"7100 North Harlem Avenue, Chicago, IL 60631, USA",CULEX PIPIENS/RESTUANS,71,N HARLEM AVE,T233,"7100 N HARLEM AVE, Chicago, IL",42.009876,-87.807277,9,17,0,"(42.009876, -87.807277)",0.630276,1,2013,165,2013-06-14
8309,2013-06-14,"7100 North Harlem Avenue, Chicago, IL 60631, USA",CULEX RESTUANS,71,N HARLEM AVE,T233,"7100 N HARLEM AVE, Chicago, IL",42.009876,-87.807277,9,2,0,"(42.009876, -87.807277)",0.630276,1,2013,165,2013-06-14
8422,2013-06-21,"7100 North Harlem Avenue, Chicago, IL 60631, USA",CULEX PIPIENS/RESTUANS,71,N HARLEM AVE,T233,"7100 N HARLEM AVE, Chicago, IL",42.009876,-87.807277,9,1,0,"(42.009876, -87.807277)",0.630276,1,2013,172,2013-06-21
8423,2013-06-21,"7100 North Harlem Avenue, Chicago, IL 60631, USA",CULEX RESTUANS,71,N HARLEM AVE,T233,"7100 N HARLEM AVE, Chicago, IL",42.009876,-87.807277,9,1,0,"(42.009876, -87.807277)",0.630276,1,2013,172,2013-06-21
8488,2013-06-27,"7100 North Harlem Avenue, Chicago, IL 60631, USA",CULEX PIPIENS/RESTUANS,71,N HARLEM AVE,T233,"7100 N HARLEM AVE, Chicago, IL",42.009876,-87.807277,9,2,0,"(42.009876, -87.807277)",0.630276,1,2013,178,2013-06-27
8489,2013-06-27,"7100 North Harlem Avenue, Chicago, IL 60631, USA",CULEX RESTUANS,71,N HARLEM AVE,T233,"7100 N HARLEM AVE, Chicago, IL",42.009876,-87.807277,9,2,0,"(42.009876, -87.807277)",0.630276,1,2013,178,2013-06-27
8664,2013-07-08,"7100 North Harlem Avenue, Chicago, IL 60631, USA",CULEX RESTUANS,71,N HARLEM AVE,T233,"7100 N HARLEM AVE, Chicago, IL",42.009876,-87.807277,9,2,0,"(42.009876, -87.807277)",0.630276,1,2013,189,2013-07-08
8846,2013-07-12,"7100 North Harlem Avenue, Chicago, IL 60631, USA",CULEX RESTUANS,71,N HARLEM AVE,T233,"7100 N HARLEM AVE, Chicago, IL",42.009876,-87.807277,9,10,0,"(42.009876, -87.807277)",0.630276,1,2013,193,2013-07-12
9023,2013-07-19,"7100 North Harlem Avenue, Chicago, IL 60631, USA",CULEX PIPIENS/RESTUANS,71,N HARLEM AVE,T233,"7100 N HARLEM AVE, Chicago, IL",42.009876,-87.807277,9,25,0,"(42.009876, -87.807277)",0.630276,1,2013,200,2013-07-19


In [ ]:
train

In [8]:
weather

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9
5,2,2007-05-03,67,48,58,M,40,50,7,0,...,HZ,M,M,M,0.00,29.46,30.12,12.9,6,13.2
6,1,2007-05-04,66,49,58,4,41,50,7,0,...,RA,0,M,0.0,T,29.31,30.05,10.4,8,10.8
7,2,2007-05-04,78,51,M,M,42,50,M,M,...,,M,M,M,0.00,29.36,30.04,10.1,7,10.4
8,1,2007-05-05,66,53,60,5,38,49,5,0,...,,0,M,0.0,T,29.40,30.10,11.7,7,12.0
9,2,2007-05-05,66,54,60,M,39,50,5,0,...,,M,M,M,T,29.46,30.09,11.2,7,11.5


In [22]:
keepme = spray[spray['Date'] == '2011-08-29']

In [65]:
spray.tail()

,Date,Time,Latitude,Longitude
14830,2013-09-05,8:34:11 PM,42.006587,-87.812355
14831,2013-09-05,8:35:01 PM,42.006192,-87.816015
14832,2013-09-05,8:35:21 PM,42.006022,-87.817392
14833,2013-09-05,8:35:31 PM,42.005453,-87.817423
14834,2013-09-05,8:35:41 PM,42.004805,-87.817460


In [64]:
keepme.tail(30)

,Date,Time,Latitude,Longitude
65,2011-08-29,7:09:18 PM,42.391270,-88.096035
66,2011-08-29,7:09:28 PM,42.391238,-88.096468
67,2011-08-29,7:09:38 PM,42.390952,-88.096437
68,2011-08-29,7:09:48 PM,42.390642,-88.096440
69,2011-08-29,7:09:58 PM,42.390335,-88.096443
70,2011-08-29,7:10:08 PM,42.390048,-88.096435
71,2011-08-29,7:10:18 PM,42.389870,-88.096258
72,2011-08-29,7:10:28 PM,42.389867,-88.095878
73,2011-08-29,7:10:38 PM,42.389858,-88.095487
74,2011-08-29,7:10:48 PM,42.389855,-88.095080


In [26]:
keepme.nunique()

Date          1
Time         95
Latitude     95
Longitude    86
dtype: int64

In [88]:
spray.Date.value_counts()

2013-08-15    2668
2013-08-29    2302
2013-07-17    2202
2011-09-07    2114
2013-07-25    1607
2013-08-22    1587
2013-08-08    1195
2013-09-05     924
2013-08-16     141
2011-08-29      95
Name: Date, dtype: int64

In [9]:
train['Species'].value_counts()

CULEX PIPIENS/RESTUANS    4752
CULEX RESTUANS            2740
CULEX PIPIENS             2699
CULEX TERRITANS            222
CULEX SALINARIUS            86
CULEX TARSALIS               6
CULEX ERRATICUS              1
Name: Species, dtype: int64

In [273]:
train['Year']=train['Date'].str.split('-').str[0].astype(int)
train['DayRunner']=pd.to_datetime(train['Date']).dt.dayofyear
train['lat_long'] = list(zip(train['Latitude'], train['Longitude']))

In [98]:
spray['Year']=spray['Date'].str.split('-').str[0].astype(int)
spray['DayRunner']=pd.to_datetime(spray['Date']).dt.dayofyear
spray['lat_long'] = list(zip(spray['Latitude'], spray['Longitude']))

In [104]:
spray_date_list = ['2013-08-15', '2013-08-29', '2013-07-17', '2011-09-07', '2013-07-25', '2013-08-22', 
                   '2013-08-08', '2013-09-05', '2013-08-16', '2011-08-29']

In [126]:
day_runner_ref = pd.DataFrame({'dates':spray_date_list})

In [128]:
day_runner_ref['DayRunner']=pd.to_datetime(day_runner_ref['dates']).dt.dayofyear

In [134]:
day_runner_ref_list = list(day_runner_ref['DayRunner'].values)

In [246]:
ref_list = list(zip(spray_date_list,day_runner_ref_list))

In [247]:
ref_list

[('2013-08-15', 227),
 ('2013-08-29', 241),
 ('2013-07-17', 198),
 ('2011-09-07', 250),
 ('2013-07-25', 206),
 ('2013-08-22', 234),
 ('2013-08-08', 220),
 ('2013-09-05', 248),
 ('2013-08-16', 228),
 ('2011-08-29', 241)]

In [191]:
spray_analysis = pd.DataFrame(columns=['Date', 'lat_long', '-10', '-9','-8','-7','-6','-5','-4','-3','-2','-1','0'
                             ,'1','2','3','4','5','6','7','8','9','10','11','12','13','14','15'
                             ,'16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33'
                             ,'34','35','36','37','38','39', '40'])

In [217]:
spray_analysis

,Date,lat_long,-10,-9,-8,-7,-6,-5,-4,-3,...,31,32,33,34,35,36,37,38,39,40


In [226]:
def analyze_spray(train, spray, ref_list, dist=1):
    spray_analysis = pd.DataFrame(columns=['Date', 'lat_long', '-10', '-9','-8','-7','-6','-5','-4','-3','-2','-1','0',
                                           '1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17',
                                           '18','19','20','21','22','23','24','25','26','27','28','29','30','31','32',
                                           '33','34','35','36','37','38','39', '40'])
    for date in ref_list:
        spray_temp = spray[spray['Date'] == date[0]]
        train_temp = train[train['Year'] == int(date[0].split('-')[0])]
        lat_long_list = list(spray_temp.lat_long.values)
        #try:
        for lat_long in lat_long_list:
            train_temp['distance_to_spray'] = train_temp['lat_long'].apply(lambda row: vincenty(row, lat_long).miles)
            train_temp_2 = train_temp[train_temp['distance_to_spray'] <= dist]
            train_temp_2['days_removed'] = train_temp_2['DayRunner'].apply(lambda x:x - date[1])
            train_temp_2 = train_temp_2[(train_temp_2['days_removed'] >= -10) & (train_temp_2['days_removed'] <= 40)]
            lat_long_list_2 = list(train_temp_2.lat_long.unique())
            for lat_long_2 in lat_long_list_2:
                train_temp_3 = train_temp_2[train_temp_2['lat_long'] == lat_long_2]
                one_row = pd.DataFrame({'Date':date[0], 
                                            'lat_long':lat_long_2, 
                                            '-10':
(train_temp_3.loc[train_temp_3['days_removed'] == -10, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == -10, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '-9':
(train_temp_3.loc[train_temp_3['days_removed'] == -9, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == -9, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '-8':
(train_temp_3.loc[train_temp_3['days_removed'] == -8, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == -8, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '-7':
(train_temp_3.loc[train_temp_3['days_removed'] == -7, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == -7, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '-6':
(train_temp_3.loc[train_temp_3['days_removed'] == -6, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == -6, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '-5':
(train_temp_3.loc[train_temp_3['days_removed'] == -5, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == -5, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '-4':
(train_temp_3.loc[train_temp_3['days_removed'] == -4, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == -4, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '-3':
(train_temp_3.loc[train_temp_3['days_removed'] == -3, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == -3, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '-2':
(train_temp_3.loc[train_temp_3['days_removed'] == -2, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == -2, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '-1':
(train_temp_3.loc[train_temp_3['days_removed'] == -1, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == -1, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '0':
(train_temp_3.loc[train_temp_3['days_removed'] == 0, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 0, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '1':
(train_temp_3.loc[train_temp_3['days_removed'] == 1, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 1, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '2':
(train_temp_3.loc[train_temp_3['days_removed'] == 2, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 2, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '3':
(train_temp_3.loc[train_temp_3['days_removed'] == 3, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 3, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '4':
(train_temp_3.loc[train_temp_3['days_removed'] == 4, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 4, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '5':
(train_temp_3.loc[train_temp_3['days_removed'] == 5, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 5, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '6':
(train_temp_3.loc[train_temp_3['days_removed'] == 6, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 6, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '7':
(train_temp_3.loc[train_temp_3['days_removed'] == 7, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 7, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '8':
(train_temp_3.loc[train_temp_3['days_removed'] == 8, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 8, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '9':
(train_temp_3.loc[train_temp_3['days_removed'] == 9, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 9, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '10':
(train_temp_3.loc[train_temp_3['days_removed'] == 10, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 10, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '11':
(train_temp_3.loc[train_temp_3['days_removed'] == 11, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 11, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '12':
(train_temp_3.loc[train_temp_3['days_removed'] == 12, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 12, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '13':
(train_temp_3.loc[train_temp_3['days_removed'] == 13, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 13, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '14':
(train_temp_3.loc[train_temp_3['days_removed'] == 14, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 14, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '15':
(train_temp_3.loc[train_temp_3['days_removed'] == 15, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 15, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '16':
(train_temp_3.loc[train_temp_3['days_removed'] == 16, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 16, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '17':
(train_temp_3.loc[train_temp_3['days_removed'] == 17, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 17, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '18':
(train_temp_3.loc[train_temp_3['days_removed'] == 18, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 18, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '19':
(train_temp_3.loc[train_temp_3['days_removed'] == 19, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 19, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '20':
(train_temp_3.loc[train_temp_3['days_removed'] == 20, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 20, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '21':
(train_temp_3.loc[train_temp_3['days_removed'] == 21, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 21, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '22':
(train_temp_3.loc[train_temp_3['days_removed'] == 22, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 22, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '23':
(train_temp_3.loc[train_temp_3['days_removed'] == 23, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 23, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '24':
(train_temp_3.loc[train_temp_3['days_removed'] == 24, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 24, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '25':
(train_temp_3.loc[train_temp_3['days_removed'] == 25, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 25, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '26':
(train_temp_3.loc[train_temp_3['days_removed'] == 26, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 26, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '27':
(train_temp_3.loc[train_temp_3['days_removed'] == 27, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 27, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '28':
(train_temp_3.loc[train_temp_3['days_removed'] == 28, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 28, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '29':
(train_temp_3.loc[train_temp_3['days_removed'] == 29, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 29, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '30':
(train_temp_3.loc[train_temp_3['days_removed'] == 30, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 30, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '31':
(train_temp_3.loc[train_temp_3['days_removed'] == 31, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 31, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '32':
(train_temp_3.loc[train_temp_3['days_removed'] == 32, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 32, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '33':
(train_temp_3.loc[train_temp_3['days_removed'] == 33, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 33, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '34':
(train_temp_3.loc[train_temp_3['days_removed'] == 34, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 34, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '35':
(train_temp_3.loc[train_temp_3['days_removed'] == 35, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 35, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '36':
(train_temp_3.loc[train_temp_3['days_removed'] == 36, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 36, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '37':
(train_temp_3.loc[train_temp_3['days_removed'] == 37, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 37, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '38':
(train_temp_3.loc[train_temp_3['days_removed'] == 38, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 38, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '39':
(train_temp_3.loc[train_temp_3['days_removed'] == 39, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 39, 'NumMosquitos'].values) != 0 else np.NaN), 
                                            '40':
(train_temp_3.loc[train_temp_3['days_removed'] == 40, 'NumMosquitos'].values[-1] if 
len(train_temp_3.loc[train_temp_3['days_removed'] == 40, 'NumMosquitos'].values) != 0 else np.NaN)})
                spray_analysis = spray_analysis.append(one_row)
        #except:
            #pass
                                        
        ref_counter += 1
    return spray_analysis

In [227]:
test = analyze_spray(train, spray, spray_date_list, day_runner_ref_list, dist=1)

/Users/christophermarker/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/christophermarker/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [232]:
test.describe()

,-1,-10,-2,-3,-4,-5,-6,-8,-9,1,...,37,38,39,4,40,5,6,8,9,lat_long
count,376.000000,0.0,0.0,0.0,0.0,9120.000000,4828.000000,376.000000,9120.00000,1728.000000,...,0.0,0.0,0.0,0.0,0.0,1846.000000,376.000000,11686.000000,1906.000000,32022.000000
mean,4.143617,NaN,NaN,NaN,NaN,26.352412,16.644988,14.946809,15.99057,17.944444,...,NaN,NaN,NaN,NaN,NaN,1.954496,5.904255,13.773575,6.947534,-22.919453
std,1.406883,NaN,NaN,NaN,NaN,17.663414,10.880383,2.271526,10.65536,0.328766,...,NaN,NaN,NaN,NaN,NaN,0.208463,3.397471,9.913545,1.373685,64.797588
min,2.000000,NaN,NaN,NaN,NaN,4.000000,3.000000,2.000000,2.00000,16.000000,...,NaN,NaN,NaN,NaN,NaN,1.000000,3.000000,2.000000,2.000000,-87.832763
25%,4.000000,NaN,NaN,NaN,NaN,4.000000,4.000000,13.000000,10.00000,18.000000,...,NaN,NaN,NaN,NaN,NaN,2.000000,4.000000,2.000000,7.000000,-87.703047
50%,4.000000,NaN,NaN,NaN,NaN,39.000000,26.000000,16.000000,14.00000,18.000000,...,NaN,NaN,NaN,NaN,NaN,2.000000,4.000000,13.000000,7.000000,-22.933136
75%,4.000000,NaN,NaN,NaN,NaN,44.000000,26.000000,16.000000,18.00000,18.000000,...,NaN,NaN,NaN,NaN,NaN,2.000000,4.750000,20.000000,7.000000,41.923738
max,15.000000,NaN,NaN,NaN,NaN,50.000000,26.000000,16.000000,42.00000,18.000000,...,NaN,NaN,NaN,NaN,NaN,2.000000,12.000000,50.000000,12.000000,42.009876


In [125]:
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,lat_long,distance_2_spray,near_spray_spot,Year,DayRunner
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,"(41.95469, -87.800991)",3.561274,0,2007,149
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,"(41.95469, -87.800991)",3.561274,0,2007,149
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0,"(41.994991, -87.769279)",2.571233,0,2007,149
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0,"(41.974089, -87.824812)",2.153478,0,2007,149
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0,"(41.974089, -87.824812)",2.153478,0,2007,149


In [193]:
train.dtypes

Date                       object
Address                    object
Species                    object
Block                       int64
Street                     object
Trap                       object
AddressNumberAndStreet     object
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
NumMosquitos                int64
WnvPresent                  int64
lat_long                   object
distance_2_spray          float64
near_spray_spot             int64
Year                        int64
DayRunner                   int64
dtype: object

In [141]:
train_test = train.copy()

In [143]:
train_test['days_removed'] = train_test['DayRunner'].apply(lambda x:x - day_runner_ref_list[0])

In [108]:
spray_test = spray[spray['Date'] == '2011-08-29']

In [116]:
gh = list(spray_test.lat_long.values)

In [151]:
for i in list(range(-10, 41)):
    print(i)

-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40


In [118]:
date1 = '2013-08-15'

In [123]:
date1 = int(date.split('-')[0])

In [124]:
date1

2013

In [99]:
spray.head()

,Date,Time,Latitude,Longitude,Year,DayRunner,lat_long
0,2011-08-29,6:56:58 PM,42.391623,-88.089163,2011,241,"(42.3916233333, -88.0891633333)"
1,2011-08-29,6:57:08 PM,42.391348,-88.089163,2011,241,"(42.3913483333, -88.0891633333)"
2,2011-08-29,6:57:18 PM,42.391022,-88.089157,2011,241,"(42.3910216667, -88.0891566667)"
3,2011-08-29,6:57:28 PM,42.390637,-88.089158,2011,241,"(42.3906366667, -88.0891583333)"
4,2011-08-29,6:57:38 PM,42.390410,-88.088858,2011,241,"(42.39041, -88.0888583333)"


In [283]:
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Year,DayRunner,lat_long
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,2007,149,"(41.95469, -87.800991)"
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,2007,149,"(41.95469, -87.800991)"
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0,2007,149,"(41.994991, -87.769279)"
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0,2007,149,"(41.974089, -87.824812)"
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0,2007,149,"(41.974089, -87.824812)"


In [ ]:
spray_date_list = 

In [100]:
spray.Date.value_counts()

2013-08-15    2668
2013-08-29    2302
2013-07-17    2202
2011-09-07    2114
2013-07-25    1607
2013-08-22    1587
2013-08-08    1195
2013-09-05     924
2013-08-16     141
2011-08-29      95
Name: Date, dtype: int64

In [205]:
test_df = pd.DataFrame({'chili':['chili1', 'chili2', 'chili1'], 
                                            'munky':['munky1', 'munky2', 'arg'], 
                                            'donkey':['donkey1', 'donkey2', np.NaN],
                                               'NUMBERS':[1,2,3
                                                    ]})

In [206]:
test_df

,NUMBERS,chili,donkey,munky
0,1,chili1,donkey1,munky1
1,2,chili2,donkey2,munky2
2,3,chili1,NaN,arg


In [190]:
test_df.isnull().sum()

chili     0
donkey    1
munky     0
dtype: int64

In [210]:
# df.loc[(df['Store Number'] == 3591) & (df['County'] == 'Webster'), 'County'] = 'Hardin'

test_df.loc[test_df['chili'] == 'chili1', 'munky'].values[-1]

'arg'

In [166]:
np.mean([10])

10.0

In [186]:
n = (test_df.loc[test_df['chili'] == 'g', 'munky'].values[-1] if 
len(test_df.loc[test_df['chili'] == 'g', 'munky'].values) != 0 else np.NaN)

In [207]:
test_df2 = test_df[(test_df['NUMBERS'] > 1) & (test_df['NUMBERS'] < 3)]

In [208]:
test_df2

,NUMBERS,chili,donkey,munky
1,2,chili2,donkey2,munky2


In [188]:
type(n)

float

In [185]:
test_df.replace('clarg',np.NaN)

,chili,donkey,munky
0,chili1,donkey1,munky1
1,chili2,donkey2,munky2
2,chili1,NaN,arg


In [225]:
list(test_df.chili.unique())

['chili1', 'chili2']

In [250]:
date = ref_list[0]

227

In [241]:
tupleguylist = [('a', 45), ('b', 46), ('c', 47)]

In [243]:
tupleguylist[1][1]

46

In [252]:
spray_temp = spray[spray['Date'] == date[0]]

In [278]:
train_temp = train[train['Year'] == int(date[0].split('-')[0])]

In [287]:
train_temp.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Year,DayRunner,lat_long,distance_to_spray
8114,2013-06-07,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,19,0,2013,158,"(41.95469, -87.800991)",0.781718
8115,2013-06-07,"1100 Roosevelt Road, Chicago, IL 60608, USA",CULEX RESTUANS,11,W ROOSEVELT,T048,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,8,2,0,2013,158,"(41.867108, -87.654224)",9.576018
8116,2013-06-07,"2200 North Cannon Drive, Chicago, IL 60614, USA",CULEX RESTUANS,22,N CANNON DR,T054,"2200 N CANNON DR, Chicago, IL",41.921965,-87.632085,8,1,0,2013,158,"(41.921965, -87.632085)",9.210220
8117,2013-06-07,"1700 West 95th Street, Chicago, IL 60643, USA",CULEX RESTUANS,17,W 95TH ST,T094,"1700 W 95TH ST, Chicago, IL",41.720848,-87.666014,9,4,0,2013,158,"(41.720848, -87.666014)",17.104709
8118,2013-06-07,"8900 South Carpenter Street, Chicago, IL 60620...",CULEX PIPIENS/RESTUANS,89,S CARPENTER ST,T159,"8900 S CARPENTER ST, Chicago, IL",41.732984,-87.649642,8,4,0,2013,158,"(41.732984, -87.649642)",16.747985


In [262]:
spray_temp.lat_long.nunique()

2668

In [276]:
spray_temp.shape

(2668, 7)

In [258]:
lat_long_list = list(spray_temp.lat_long.values)

In [259]:
lat_long_list

[(41.944718333333299, -87.8081866666667),
 (41.944125, -87.808176666666697),
 (41.943768333333296, -87.808161666666692),
 (41.943404999999998, -87.808153333333308),
 (41.942983333333295, -87.808134999999993),
 (41.942518333333297, -87.808125000000004),
 (41.941740000000003, -87.808099999999996),
 (41.941406666666701, -87.8080966666667),
 (41.941020000000002, -87.808080000000004),
 (41.940581666666702, -87.808075000000002),
 (41.940109999999997, -87.808056666666701),
 (41.939638333333299, -87.8080383333333),
 (41.939225, -87.808014999999997),
 (41.938793333333294, -87.808006666666699),
 (41.9385616666667, -87.8081283333333),
 (41.9385616666667, -87.808630000000008),
 (41.938556666666706, -87.808936666666696),
 (41.938773333333295, -87.809173333333305),
 (41.939176666666704, -87.809190000000001),
 (41.939476666666707, -87.809193333333297),
 (41.9398816666667, -87.809201666666695),
 (41.940788333333295, -87.8092616666667),
 (41.941231666666702, -87.809271666666703),
 (41.941668333333297, 

In [253]:
spray_temp.shape

(2668, 7)

In [286]:
train_temp['distance_to_spray'] = train_temp['lat_long'].apply(lambda row: vincenty(row, lat_long_list[0]).miles)

/Users/christophermarker/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [326]:
spray_analysis

,1-7_days_after_avg,10_days_prior_avg,15-21_days_after_avg,22-28_days_after_avg,29-35_days_after_avg,36-42_days_after_avg,8-14_days_after_avg,Date,lat_long
0,23.0,32.666667,NaN,NaN,13.5,23.333333,23.333333,2013-08-15,41.954690
1,23.0,32.666667,NaN,NaN,13.5,23.333333,23.333333,2013-08-15,-87.800991


In [291]:
lat_long_list_2

(41.954689999999999, -87.800990999999996)

In [297]:
train_temp_3

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Year,DayRunner,lat_long,distance_to_spray,days_removed
9372,2013-08-08,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,27,1,2013,220,"(41.95469, -87.800991)",0.781718,-7
9373,2013-08-08,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,50,0,2013,220,"(41.95469, -87.800991)",0.781718,-7
9374,2013-08-08,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,50,1,2013,220,"(41.95469, -87.800991)",0.781718,-7
9375,2013-08-08,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,13,0,2013,220,"(41.95469, -87.800991)",0.781718,-7
9376,2013-08-08,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,43,1,2013,220,"(41.95469, -87.800991)",0.781718,-7
9377,2013-08-08,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,13,1,2013,220,"(41.95469, -87.800991)",0.781718,-7
9553,2013-08-15,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,6,0,2013,227,"(41.95469, -87.800991)",0.781718,0
9554,2013-08-15,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,2013,227,"(41.95469, -87.800991)",0.781718,0
9555,2013-08-15,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,4,0,2013,227,"(41.95469, -87.800991)",0.781718,0
9710,2013-08-22,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,34,1,2013,234,"(41.95469, -87.800991)",0.781718,7


In [303]:
np.mean(train_temp_3[(train_temp_3['days_removed'] >= 15) & (train_temp_3['days_removed'] <= 21)].NumMosquitos.values)

23.0

In [327]:
for x, y in lat_long_list_2[:2]:
    print(x,y)

41.95469 -87.800991


In [329]:
dist=1
date = ref_list[0]
spray_analysis = pd.DataFrame(columns=['Date', 'lat_long', '10_days_prior_avg','1-7_days_after_avg',
                                       '8-14_days_after_avg','15-21_days_after_avg','22-28_days_after_avg',
                                       '29-35_days_after_avg','36-42_days_after_avg'])
spray_temp = spray[spray['Date'] == date[0]]
train_temp = train[train['Year'] == int(date[0].split('-')[0])]
lat_long_list = list(spray_temp.lat_long.values)
train_temp['distance_to_spray'] = train_temp['lat_long'].apply(lambda row: vincenty(row, lat_long_list[0]).miles)
train_temp_2 = train_temp[train_temp['distance_to_spray'] <= dist]
train_temp_2['days_removed'] = train_temp_2['DayRunner'].apply(lambda x:x - date[1])
train_temp_2 = train_temp_2[(train_temp_2['days_removed'] >= -10) & (train_temp_2['days_removed'] <= 42)]
lat_long_list_2 = list(train_temp_2.lat_long.unique())
for x, y in lat_long_list_2:
#     print(lat_long_2)
#     print(train_temp_2.iloc[0]['lat_long'])
#     print(train_temp_2[train_temp_2['lat_long'] == lat_long_2] )
    train_temp_3 = train_temp_2[train_temp_2['lat_long'] == (x,y)]  
    one_row = pd.DataFrame({'Date':date[0], 'lat_long':(x,y), 
    '10_days_prior_avg':np.mean(train_temp_3[(train_temp_3['days_removed'] >= -10) & (train_temp_3['days_removed'] < 0)].NumMosquitos.values),
    '1-7_days_after_avg':np.mean(train_temp_3[(train_temp_3['days_removed'] >= 1) & (train_temp_3['days_removed'] <= 7)].NumMosquitos.values),
    '8-14_days_after_avg':np.mean(train_temp_3[(train_temp_3['days_removed'] >= 8) & (train_temp_3['days_removed'] <= 14)].NumMosquitos.values),
    '15-21_days_after_avg':np.mean(train_temp_3[(train_temp_3['days_removed'] >= 15) & (train_temp_3['days_removed'] <= 21)].NumMosquitos.values),
    '22-28_days_after_avg':np.mean(train_temp_3[(train_temp_3['days_removed'] >= 28) & (train_temp_3['days_removed'] <= 14)].NumMosquitos.values),
    '29-35_days_after_avg':np.mean(train_temp_3[(train_temp_3['days_removed'] >= 29) & (train_temp_3['days_removed'] <= 35)].NumMosquitos.values),
    '36-42_days_after_avg':np.mean(train_temp_3[(train_temp_3['days_removed'] >= 8) & (train_temp_3['days_removed'] <= 14)].NumMosquitos.values)})                        

    spray_analysis = spray_analysis.append(one_row)

/Users/christophermarker/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/christophermarker/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/christophermarker/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/christophermarker/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encounte

In [330]:
spray_analysis

,1-7_days_after_avg,10_days_prior_avg,15-21_days_after_avg,22-28_days_after_avg,29-35_days_after_avg,36-42_days_after_avg,8-14_days_after_avg,Date,lat_long
0,23.0,32.666667,NaN,NaN,13.5,23.333333,23.333333,2013-08-15,41.954690
1,23.0,32.666667,NaN,NaN,13.5,23.333333,23.333333,2013-08-15,-87.800991


In [315]:
ref_list

[('2013-08-15', 227),
 ('2013-08-29', 241),
 ('2013-07-17', 198),
 ('2011-09-07', 250),
 ('2013-07-25', 206),
 ('2013-08-22', 234),
 ('2013-08-08', 220),
 ('2013-09-05', 248),
 ('2013-08-16', 228),
 ('2011-08-29', 241)]

In [ ]:
def analyze_spray(train, spray, ref_list, dist=1):
    spray_analysis = pd.DataFrame(columns=['Date', 'lat_long', '10_days_prior_avg','1-7_days_after_avg',
                                       '8-14_days_after_avg','15-21_days_after_avg','22-28_days_after_avg',
                                       '29-35_days_after_avg','36-42_days_after_avg'])
    for date in ref_list:
        spray_temp = spray[spray['Date'] == date[0]]
        train_temp = train[train['Year'] == int(date[0].split('-')[0])]
        lat_long_list = list(spray_temp.lat_long.values)
        #try:
        for lat_long in lat_long_list:
            train_temp['distance_to_spray'] = train_temp['lat_long'].apply(lambda row: vincenty(row, lat_long).miles)
            train_temp_2 = train_temp[train_temp['distance_to_spray'] <= dist]
            train_temp_2['days_removed'] = train_temp_2['DayRunner'].apply(lambda x:x - date[1])
            train_temp_2 = train_temp_2[(train_temp_2['days_removed'] >= -10) & (train_temp_2['days_removed'] <= 42)]
            lat_long_list_2 = list(train_temp_2.lat_long.unique())
            for lat_long_2 in lat_long_list_2:
                train_temp_3 = train_temp_2[train_temp_2['lat_long'] == lat_long_2]
                one_row = pd.DataFrame({'Date':date[0], 'lat_long':lat_long_2, 
    '10_days_prior_avg':np.mean(train_temp_3[(train_temp_3['days_removed'] >= -10) & (train_temp_3['days_removed'] < 0)].NumMosquitos.values),
    '1-7_days_after_avg':np.mean(train_temp_3[(train_temp_3['days_removed'] >= 1) & (train_temp_3['days_removed'] <= 7)].NumMosquitos.values),
    '8-14_days_after_avg':np.mean(train_temp_3[(train_temp_3['days_removed'] >= 8) & (train_temp_3['days_removed'] <= 14)].NumMosquitos.values),
    '15-21_days_after_avg':np.mean(train_temp_3[(train_temp_3['days_removed'] >= 15) & (train_temp_3['days_removed'] <= 21)].NumMosquitos.values),
    '22-28_days_after_avg':np.mean(train_temp_3[(train_temp_3['days_removed'] >= 28) & (train_temp_3['days_removed'] <= 14)].NumMosquitos.values),
    '29-35_days_after_avg':np.mean(train_temp_3[(train_temp_3['days_removed'] >= 29) & (train_temp_3['days_removed'] <= 35)].NumMosquitos.values),
    '36-42_days_after_avg':np.mean(train_temp_3[(train_temp_3['days_removed'] >= 8) & (train_temp_3['days_removed'] <= 14)].NumMosquitos.values)})                        

                spray_analysis = spray_analysis.append(one_row)